In [1]:
import pandas as pd
from phasepy import mixture, component, virialgamma, preos
from phasepy.equilibrium import tpd_min, tpd_minimas, lle_init
import itertools
import numpy as np
from scipy.stats import qmc
import matplotlib.pyplot as plt
import csv
import time

In [2]:
methanol = component(name='methanol', Tc=512.5, Pc=80.84, Zc=0.222, Vc=117, GC={'CH3OH':1})
ethanol = component(name='ethanol', Tc=514, Pc=61.37, Zc=0.241, Vc=168, GC={'CH3':1, 'CH2':1, 'OH':1})
_2propanol = component(name='2-propanol', Tc=508.3, Pc=47.65, Zc=0.25, Vc=222, GC={'CH3':2, 'CH':1, 'OH':1})
acetone = component(name='acetone', Tc=508.1, Pc=47, Zc=0.237, Vc=213, GC={'CH3CO':1, 'CH3':1})
mibk = component(name='mibk', Tc=574.6, Pc=32.7, Zc=0.256, Vc=317, GC={'CH3CO':1, 'CH3':2, 'CH2':1, 'CH':1})
ethac = component(name='ethyl-acetate', Tc=523.3, Pc=38.8, Zc=0.255, Vc=286, GC={'CH3COO':1, 'CH3':1, 'CH2':1})
chloroform = component(name='chloroform', Tc=536.4, Pc=54.72, Zc=0.293, Vc=239, GC={'CHCL3':1})
toluene = component(name='toluene', Tc=591.75, Pc=41.08, Zc=0.264, Vc=316, GC={'ACH':5, 'ACCH3':1})
water = component(name='water', Tc=647.096, Pc=220.64, Zc=0.229, Vc=55.9472,GC={'H2O':1})
ibuprofen = component(name='ibuprofen', Tc=765, Pc=29.8, Zc=0.313, Vc=668,
                      GC={'CH3':3, 'CH':1, 'ACH':4, 'ACCH2':1, 'ACCH':1, 'COOH':1},
                      dHf=25500, Tf=347.15)

In [3]:
component_map = {"methanol" : methanol, "ethanol" : ethanol, "2-propanol" : _2propanol, "acetone" : acetone,
                 "mibk" : mibk, "ethyl-acetate": ethac, "chloroform" : chloroform, "toluene": toluene,
                 "water" : water}

In [4]:
component_list = [methanol,ethanol,_2propanol,acetone,mibk,ethac,chloroform,toluene,water]
binary_mix = itertools.combinations(component_list, 2)

In [5]:
def return_eos(c1,c2):
    mix_func = mixture(c1,c2)
    mix_func += ibuprofen
    mix_func.original_unifac()
    eos = virialgamma(mix_func, virialmodel='ideal_gas', actmodel='original_unifac')
    return eos

In [6]:
%%time
eos_dict = {}
counter=0
for i in binary_mix:
    eos = return_eos(i[0],i[1])
    eos_dict[counter] = eos
    counter+=1

CPU times: total: 3.81 s
Wall time: 4.21 s


In [7]:
df = pd.read_csv('Case1.csv')

In [8]:
df_part = pd.read_csv('Case1.csv', nrows=512)
z1 = df_part['x1']
z2 = df_part['x2']
z3 = 1 - z1 - z2
z = np.column_stack((z1, z2, z3))
T = df_part['T']

In [9]:
st = time.time()  # start timer
nmin = 3
P = 1.01325

ismisc_list = []
for i in eos_dict:
    for j in range(512):
        tpd_m = tpd_minimas(nmin, z[j], T[j], P, eos_dict[i], stateW='L', stateZ='L')
        tpd_values = tpd_m[-1]  # outputs nmin TPD values
        min_tpd_value = np.min(tpd_values, axis=0)  # minimum TPD values
        ismisc = np.where(min_tpd_value < -1e-6, 0, 1)
        ismisc_list.append(ismisc)
    #min_tpd_value_index = np.argmin(tpd_values, axis=0)  # indexes the minimum TPD values
    #tpd_m_w = tpd_m[0][np.arange(len(df)), min_tpd_value_index].tolist()  # converts type from array to list

    #master_list = np.column_stack((T, mix_func.names[0], mix_func.names[1], z1, z2, min_tpd_value, ismisc)).tolist()

et = time.time()
elapsed_time = et - st
print('Execution time:', elapsed_time, 'seconds')  # execution time

Execution time: 370.4925887584686 seconds


In [10]:
print(len(ismisc_list))

18432


In [11]:
df['ismisc']=ismisc_list

In [12]:
df.to_csv('Case1_tpd_calc_lle.csv')